In [2]:
class State:
    """ This is a class to define the properties of an individual State in a automata"""

    def __init__(self, name, FinalState):
        """ Initailize the State with Arguement:
            ID           -  Unique Identifier for the State (String or Integer)
            StateType    -  Defines the if the State is Terminal State or vice versa (Boolean)
            Transition   -  COntains all transitions from the State (Dictionary)"""

        self.ID = name
        self.StateType = FinalState
        self.Transition = dict()

    def addTransition(self, InString, OutString, OutState):
        """ Appends a Transition into the class variable 'Transition'
            Arguement of the method:
            Instring    -  Transistion string input 
            Outstring   -  Output string of the transition
            Outstate    -  Next state of the transition
            
            Return:
            None
            """

        self.Transition[InString] = Transition(self, InString, OutString, OutState)   


In [3]:
class Transition:
    """The Basic Transition implementation of a state is defined"""

    def __init__(self, InState, InString, OutState, OutString = None):
        """ Defines the basic transition properties and intialized with Argument
            Instate    -   Current state ID
            Instring   -   Transition string input
            OutString  -   Transition output sting
            OutState   -   Next state's ID of the transition
        """

        self.InState = InState
        self.InString = InString
        self.OutString = OutString
        self.OutState = OutState

In [4]:
class Automata:
    """ Basic class to define the automata's structure and its transitions. Further this automata model and be deployed in acceptor or automaton for applications."""

    def __init__(self, StateCount, Alphabet, OutAlphabets, InitialStaeCount, FinalStaeCount):
        
        self.StatesCount = StateCount
        self.Alphabet = Alphabet
        self.OutAlphabets = OutAlphabets
        self.InitialStaeCount = InitialStaeCount
        self.FinalStaeCount = FinalStaeCount

        self.States = dict()
    
    # Methods for building the automata structure

    def AddState(self, ID, Type):

        if self.StatesCount < len(self.States):
            self.States[ID] = State(ID, Type)
        else:
            print("Exceeding the number of states!!!")

    def AddStateTransition(self, InState, Instring, OutString, OutState):

        if InState in self.States.keys() and OutState in self.States.keys():
            In = self.States[InState]
            In.addTransition(Instring, OutString, OutState)
            return 1
        else:
            return -1

    # Create a new Transition object for a state

    def AddsetTransition(self, InState, set ,OutState):
        
        for a in set:
            PCode = self.AddStateTransition(InState, a, a, OutState)
            if PCode == -1:
                print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
                break
        
    def AddsetEpilsonTransition(self, InState, set ,OutState):

        for a in set:
            PCode = self.AddStateTransition(InState, a, '', OutState)
            if PCode == -1:
                print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
                break
  
    def AddEpilsonTransition(self, InState, OutState):

        PCode = self.AddStateTransition(InState, '', '', OutState)
        if PCode == -1:
            print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')

    def AddEpilsonStringTransition(self, InState, OutString, OutState):

        PCode = self.AddStateTransition(InState, '', OutString, OutState)
        if PCode == -1:
            print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')


In [5]:
Q0 = State('A', False)
Q0.addTransition('a', '', Q0)
Q0.addTransition('b', '', Q0)
Q0.Transition

{'a': <__main__.Transition at 0x7fcb7428f3d0>,
 'b': <__main__.Transition at 0x7fcb743831c0>}